## ETL simple que implica extraer datos de una fuente, aplicar una transformación con Python y cargar los datos en una tabla de Snowflake.(herrameintas databricks, python, Snow SQL)

In [ ]:
# Importar bibliotecas necesarias
import snowflake.connector
from pyspark.sql import SparkSession

# Inicializar la sesión de Spark
spark = SparkSession.builder.appName("ETL").getOrCreate()

# Configurar las credenciales de Snowflake
snowflake_options = {
    "sfURL": "https://your-account.snowflakecomputing.com",
    "sfDatabase": "your-database",
    "sfWarehouse": "your-warehouse",
    "sfRole": "your-role",
    "sfSchema": "your-schema",
    "sfWarehouse": "your-warehouse",
    "sfDatabase": "your-database",
    "sfWarehouse": "your-warehouse",
    "sfRole": "your-role"
}

# Definir una consulta SQL para extraer datos
query = """
    SELECT column1, column2
    FROM your_source_table
    WHERE date_column >= '2023-01-01'
"""

# Cargar datos desde Snowflake a Spark DataFrame
df = spark.read.format("snowflake").options(**snowflake_options).option("query", query).load()

# Realizar transformaciones en los datos (por ejemplo, filtrar, agregar columnas, etc.)
df_transformed = df.filter(df["column1"] > 100)

# Conectar a Snowflake y cargar los datos transformados en una tabla
with snowflake.connector.connect(**snowflake_options) as conn:
    with conn.cursor() as cursor:
        cursor.execute("USE WAREHOUSE {}".format(snowflake_options["sfWarehouse"]))
        cursor.execute("USE DATABASE {}".format(snowflake_options["sfDatabase"]))
        cursor.execute("USE SCHEMA {}".format(snowflake_options["sfSchema"]))
        cursor.execute("TRUNCATE TABLE your_target_table")
        df_transformed.write.mode("append").format("snowflake").options(**snowflake_options).save()

# Detener la sesión de Spark
spark.stop()
